In [5]:
%run load_pga_tee_time.py
df_tee_time = df

   Group Number            Tee Time  Start Tee  Group Location  Group Hole  \
0             1 2025-03-20 11:35:00          1  Round Complete          18   
1             1 2025-03-20 11:35:00          1  Round Complete          18   

                             Course Name Group Status    Player Name Country  
0  Innisbrook Resort (Copperhead Course)     COMPLETE  Danny Willett     ENG  
1  Innisbrook Resort (Copperhead Course)     COMPLETE      Rico Hoey     PHI  


In [6]:
import pandas as pd

# User input for the date
# Need try catch if date is old tournament.
date_input = '2025-03-22'

# Convert user input to a pandas Timestamp
date_input = pd.to_datetime(date_input)

# Filter the DataFrame to include only rows with the specified date
df_tee_time = df_tee_time[df_tee_time['Tee Time'].dt.date == date_input.date()]

print(df_tee_time)


     Group Number            Tee Time  Start Tee Group Location  Group Hole  \
306             1 2025-03-22 11:30:00          1    Coming soon           1   
307             1 2025-03-22 11:30:00          1    Coming soon           1   
308             2 2025-03-22 11:39:00          1    Coming soon           1   
309             2 2025-03-22 11:39:00          1    Coming soon           1   
310             3 2025-03-22 11:48:00          1    Coming soon           1   
..            ...                 ...        ...            ...         ...   
379            37 2025-03-22 17:40:00          1    Coming soon           1   
380            38 2025-03-22 17:50:00          1    Coming soon           1   
381            38 2025-03-22 17:50:00          1    Coming soon           1   
382            39 2025-03-22 18:00:00          1    Coming soon           1   
383            39 2025-03-22 18:00:00          1    Coming soon           1   

                               Course Name Group St

In [7]:
%run load_pga_tournament.py
df_tournament = df

  POS        PLAYER TOT THRU    R1    R2 R3 R4  total_stroke
0   1  J. Bridgeman  -6    F  67.0  69.0  -  -           136
1  T2    V. Hovland  -5   F*  70.0  67.0  -  -           137


C:\Users\rypi3\source\repos\data_golf\load_pga_tournament.py:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('-', inplace=True)


In [8]:
%run load_pga_fedexcup_rank.py
df_rank = df

    Player Name           Country Projected Rank Official Rank  \
0  Rory McIlroy  Northern Ireland              1             1   
1   Sepp Straka           Austria              2             2   

  Point Data Projected Point Data Official  
0            1,586.286               1,586  
1            1,328.637               1,283  


In [9]:
# Function to extract last name
def extract_last_name(full_name):
    return full_name.split()[-1]

In [10]:
def extract_name_with_initial(full_name):
    name_parts = full_name.split()
    first_initial = name_parts[0][0] + '.' if name_parts else ''
    last_name = name_parts[-1] if len(name_parts) > 1 else ''
    return f"{first_initial} {last_name}"

In [11]:
# Apply the function to the Full Name column to create a new Last Name column
df_tournament['Name_Key'] = df_tournament['PLAYER'].apply(extract_last_name)
df_tee_time['Name_Key'] = df_tee_time['Player Name'].apply(extract_last_name)
df_rank['Name_Key'] = df_rank['Player Name'].apply(extract_last_name)

C:\Users\rypi3\AppData\Local\Temp\ipykernel_9044\4203428155.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tee_time['Name_Key'] = df_tee_time['Player Name'].apply(extract_last_name)


In [12]:
# Apply the function to the Full Name column to create a new Last Name column
df_tournament['Name_Key2'] = df_tournament['PLAYER'].apply(extract_name_with_initial)
df_tee_time['Name_Key2'] = df_tee_time['Player Name'].apply(extract_name_with_initial)
df_rank['Name_Key2'] = df_rank['Player Name'].apply(extract_name_with_initial)

C:\Users\rypi3\AppData\Local\Temp\ipykernel_9044\643551268.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tee_time['Name_Key2'] = df_tee_time['Player Name'].apply(extract_name_with_initial)


In [13]:
df_tee_time.tail(2)

,Group Number,Tee Time,Start Tee,Group Location,Group Hole,Course Name,Group Status,Player Name,Country,Name_Key,Name_Key2
382,39,2025-03-22 18:00:00,1,Coming soon,1,Innisbrook Resort (Copperhead Course),NOT_STARTED,Jacob Bridgeman,USA,Bridgeman,J. Bridgeman
383,39,2025-03-22 18:00:00,1,Coming soon,1,Innisbrook Resort (Copperhead Course),NOT_STARTED,Viktor Hovland,NOR,Hovland,V. Hovland


In [14]:
df_times_rank = pd.merge(df_tee_time, df_tournament, on='Name_Key2', how='left')
df_times_rank = pd.merge(df_times_rank, df_rank, on='Name_Key2', how='left')

In [15]:
print(df_rank.shape[1] + df_tee_time.shape[1] + df_tournament.shape[1])

30


In [16]:
df_times_rank = df_times_rank[['Group Number','Tee Time', 'Player Name_x','POS','Official Rank']]

In [18]:
# Convert numeric columns to integer
# Replace '-', remove 'T', handle NaN values, and safely convert to integers
df_times_rank['Official Rank'] = (
    df_times_rank['Official Rank']
    .replace('-', 1000)               # Replace '-' with 1000
    .astype(str)                      # Convert the column to string
    .str.replace('T', '', regex=False)  # Remove 'T' (regex=False makes it explicit)
    .replace('nan', 1000)             # Handle 'nan' represented as a string
    .fillna(1000)                     # Handle actual NaN values
    .astype(float)                    # Convert to float first
    .astype(int)                      # Finally, convert to integer
)



In [19]:
df_times_rank.dtypes

Group Number              int64
Tee Time         datetime64[ns]
Player Name_x            object
POS                      object
Official Rank             int64
dtype: object

In [20]:
# Group by the key and calculate the difference between rows
df_times_rank['Difference'] = df_times_rank.groupby('Group Number')['Official Rank'].diff().fillna(0)

In [21]:
print(f'original row count: {df_times_rank.shape[0]}')

original row count: 78


In [22]:
# Remove duplicates from upcoming tee times
df_times_rank.drop_duplicates(inplace=True)

In [23]:
print(f'original row count: {df_times_rank.shape[0]}')


original row count: 78


In [24]:
df_best_groups = df_times_rank[df_times_rank['Difference']>100]
df_best_groups.head()

,Group Number,Tee Time,Player Name_x,POS,Official Rank,Difference
19,10,2025-03-22 12:51:00,Braden Thornberry,T53,1000,922.0
21,11,2025-03-22 13:05:00,Luke Clanton,T53,1000,874.0
27,14,2025-03-22 13:35:00,Kaito Onishi,T39,1000,853.0
33,17,2025-03-22 14:05:00,Kevin Kisner,T39,1000,840.0
47,24,2025-03-22 15:20:00,David Lipsky,T26,183,147.0


In [25]:
# Filter rows in df_other based on Group Numbers in df_best_groups
df_bet = df_times_rank[df_times_rank['Group Number'].isin(df_best_groups['Group Number'])]

df_bet.head(50)


,Group Number,Tee Time,Player Name_x,POS,Official Rank,Difference
18,10,2025-03-22 12:51:00,Ryan Gerard,T53,78,0.0
19,10,2025-03-22 12:51:00,Braden Thornberry,T53,1000,922.0
20,11,2025-03-22 13:05:00,Matt McCarty,T53,126,0.0
21,11,2025-03-22 13:05:00,Luke Clanton,T53,1000,874.0
26,14,2025-03-22 13:35:00,C.T. Pan,T39,147,0.0
27,14,2025-03-22 13:35:00,Kaito Onishi,T39,1000,853.0
32,17,2025-03-22 14:05:00,Chandler Phillips,T39,160,0.0
33,17,2025-03-22 14:05:00,Kevin Kisner,T39,1000,840.0
46,24,2025-03-22 15:20:00,Andrew Novak,T26,36,0.0
47,24,2025-03-22 15:20:00,David Lipsky,T26,183,147.0


In [26]:
df_bet['Player Name_x'][df_bet['Difference'] == 0]

18          Ryan Gerard
20         Matt McCarty
26             C.T. Pan
32    Chandler Phillips
46         Andrew Novak
72          Shane Lowry
Name: Player Name_x, dtype: object

In [27]:
import tabulate

ModuleNotFoundError: No module named 'tabulate'